In [3]:
import pandas as pd
import numpy as np
import lightgbm as lgbm
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import auc
import random
import os
import ast

In [4]:
!pip install feature_engine

     |████████████████████████████████| 8.7MB 22.2MB/s 
  Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
cd /content/drive/My Drive/kalapa

/content/drive/My Drive/kalapa


In [9]:
train_df = pd.read_csv('/content/drive/My Drive/kalapa/dataset/train.csv')
test_df = pd.read_csv('/content/drive/My Drive/kalapa/dataset/test.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (35,43) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (34,42) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
train_df.head()

,id,label,Field_1,Field_2,Field_3,Field_4,Field_5,Field_6,Field_7,Field_8,Field_9,Field_10,Field_11,Field_12,Field_13,Field_14,Field_15,Field_16,Field_17,Field_18,Field_19,Field_20,Field_21,Field_22,Field_23,Field_24,Field_25,Field_26,Field_27,Field_28,Field_29,Field_30,Field_31,Field_32,Field_33,Field_34,Field_35,ngaySinh,namSinh,gioiTinh,...,partner2_H,partner2_K,partner2_L,partner3_A,partner3_B,partner3_C,partner3_D,partner3_E,partner3_F,partner3_G,partner3_H,partner3_K,partner3_L,partner4_A,partner4_B,partner4_C,partner4_D,partner4_E,partner4_F,partner4_G,partner4_H,partner4_K,partner4_L,partner5_A,partner5_B,partner5_C,partner5_D,partner5_E,partner5_F,partner5_G,partner5_H,partner5_K,partner5_L,brief,num_of_phone,Field_78,Field_79,Field_80,Field_81,Field_82
0,0,1,2019-07-24T03:39:02.854Z,2019-07-31T20:10:02Z,1.0,GH,2018-12-27,2018-12-27,2019-01-01,2019-07-31,2018-12-27,1.0,2019-07-31,G8,1.0,1.0,NaN,1.0,1.0,Trung tâm Kinh doanh tiền mặt,0.0,4258600.0,4.5,1.0,16.0,1.0,2018-12-27,1.0,0.0,0.0,0.0,1.0,1.0,2018-12-28,2022-09-01,20180107,2019-07-31,19910615.0,0.0,MALE,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,cb1,1.0,NaN,NaN,NaN,NaN,1
1,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,1.0,NaN,NaN,NaN,NaN,1
2,2,0,2019-01-17T07:15:26.367Z,2019-01-17T07:17:45Z,2.0,T1,2019-01-17,2019-01-17,2019-01-01,2019-12-31,2019-01-17,1.0,NaN,NaN,1.0,1.0,2019-01-17,1.0,1.0,NaN,0.0,5000000.0,4.5,1.0,10.0,1.0,2019-01-17,1.0,0.0,0.0,0.0,1.0,1.0,2019-01-17,2023-03-01,20190102,NaN,19980601.0,0.0,FEMALE,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1.0,33.0,10.769445,6.466667,0.0,2
3,3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1.0,33.0,10.769445,6.466667,0.0,3
4,4,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1.0,33.0,10.769445,6.466667,0.0,3


In [11]:
def get_new_columns(name, aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]

def agg_features(df, groupby, agg, prefix):
    agg_df = df.groupby(groupby).agg(agg)
    agg_df.columns = get_new_columns(prefix, agg)
    return agg_df

In [13]:
train_df['Field_36'].replace('75', 'None', inplace=True)
train_df['Field_36'].replace('79', 'None', inplace=True)
train_df['Field_36'].replace('80', 'None', inplace=True)
train_df['Field_36'].replace('86', 'None', inplace=True)

test_df['Field_36'].replace('75', 'None', inplace=True)
test_df['Field_36'].replace('79', 'None', inplace=True)
test_df['Field_36'].replace('80', 'None', inplace=True)
test_df['Field_36'].replace('86', 'None', inplace=True)

In [14]:
train_df["null_sum"] = train_df.isnull().sum(axis=1)
test_df["null_sum"] = test_df.isnull().sum(axis=1)

In [16]:
def clean_9(x):
    if x == "na":
        return "None"
    else:
        return x
def clean_33(x):
    if x == "Zero":
        return 0
    elif x == "One":
        return 1
    elif x == "Two":
        return 2
    elif x == "Three":
        return 3
    elif x == "Four":
        return 4
    else:
        return x
    
def clean_62(x):
    if x == "I":
        return 1
    elif x == "II":
        return 2
    elif x == "III":
        return 3
    elif x == "IV":
        return 4
    elif x == "V":
        return 5
    else:
        return x
    
    
def clean_42(x):
    if x == "Zezo":
        return 0
    elif x == "One":
        return 1
    else:
        return x
    
def clean_43(x):
    if x == "B":
        return 1
    elif x == "C":
        return 2
    elif x == "D":
        return 3
    elif x == "E":
        return 4
    elif x == "F":
        return 5
    elif x == "G":
        return 6
    elif x == "H":
        return 7
    elif x == "I":
        return 8
    else:
        return x
    
def clean_44(x):
    if x == "One":
        return 1
    elif x == "Two":
        return 2
    else:
        return x
        
    
def clean_45(x):
    if x == 1.0:
        return "1"
    elif x == 2.0:
        return "2"
    elif x == 3.0:
        return "3"
    else:
        return x

def fix_true_false(x):
    if x == "TRUE":
        return True
    elif x == "FALSE":
        return False
    elif x == "None":
        return np.nan
    else:
        return x
    
def clean_11(x):
    if x == "None":
        return np.nan
    else:
        return float(x)

In [18]:
train_df["Field_36"] = train_df["Field_36"].apply(clean_9)
train_df["Field_62"] = train_df["Field_62"].apply(clean_62)
train_df["Field_47"] = train_df["Field_47"].apply(clean_42)
train_df["Field_66"] = train_df["Field_66"].apply(clean_43)
#train_df["C_numOrg"] = train_df["FIELD_44"].apply(clean_44)
#train_df["FIELD_45"] = train_df["FIELD_45"].apply(clean_45)

test_df["Field_36"] = train_df["Field_36"].apply(clean_9)
test_df["Field_62"] = train_df["Field_62"].apply(clean_62)
test_df["Field_47"] = train_df["Field_47"].apply(clean_42)
test_df["Field_66"] = train_df["Field_66"].apply(clean_43)
#train_df["C_numOrg"] = train_df["FIELD_44"].apply(clean_44)
#train_df["FIELD_45"] = train_df["FIELD_45"].apply(clean_45)

In [19]:
cat_cols = [
    "province", "district", 
    "maCv",
    "maCv2",
    "FIELD_18", "FIELD_19", "FIELD_20", "FIELD_25", "FIELD_26", "FIELD_27", "FIELD_28", "FIELD_38",

    
    "FIELD_8",
    
    "FIELD_12",
    "FIELD_17",
    "FIELD_29",
    "FIELD_42",
    "FIELD_43",
    "FIELD_44",
    "FIELD_45",
    "FIELD_9", 
    
    "FIELD_10", 
    "FIELD_13", 
    "FIELD_24", 
    "FIELD_30", 
    "FIELD_31", 
    "FIELD_35", 
    "FIELD_36", 
    "FIELD_37"
]

num_cols = [
 'age_source1',
 'age_source2',
 'FIELD_1',
 'FIELD_2',
 'FIELD_4',
 'FIELD_5',
 'FIELD_6',
 'FIELD_21',
 "FIELD_11",
 'FIELD_22',
 'FIELD_32',
 'FIELD_33',
 'FIELD_34',
 'FIELD_46',
 'FIELD_50',
 'FIELD_51',
 'FIELD_52',
 'FIELD_53',
 'FIELD_54',
 'FIELD_55',
 'FIELD_56',
 'FIELD_57'
]

NameError: ignored

In [ ]:
    'id', "FIELD_7", "FIELD_23", "FIELD_3",
    "FIELD_39", "FIELD_40",
    "FIELD_41", "FIELD_42", "FIELD_43", "FIELD_44", "FIELD_45",
    'FIELD_4',
    'FIELD_5',
    'FIELD_6',
    "FIELD_17", "FIELD_24", "FIELD_35",
    "FIELD_48",
    "FIELD_19",
    "FIELD_20",
    "FIELD_54", "FIELD_18",
    "FIELD_49", "FIELD_25",
    'FIELD_57', 'FIELD_18_fe', 'FIELD_26_fe', 'FIELD_26', 'FIELD_10',
    'FIELD_10_fe', 'FIELD_36_fe', 'FIELD_24_fe', 'FIELD_12_fe',
    'FIELD_12', 'FIELD_36', 'FIELD_31', 'FIELD_34', 'FIELD_33',
    'FIELD_31_fe', 'FIELD_37', 'FIELD_37_fe', 'FIELD_29',
    'FIELD_29_fe', 'FIELD_32', 'FIELD_27', 'FIELD_27_fe', 'FIELD_28',
    'FIELD_15', 'FIELD_14', 'FIELD_28_fe', 'FIELD_38', 'FIELD_38_fe',
    'FIELD_21', 'FIELD_42_fe',
        
#     'FIELD_53', 'FIELD_1', 'null_sum', 'FIELD_2', 'FIELD_30',
    'FIELD_30_fe', 'FIELD_43_fe',
    'FIELD_17_fe', 'FIELD_50', 
    "null_sum",
   'FIELD_56', 'FIELD_8', 'FIELD_8_fe', 'FIELD_20_fe', 'FIELD_19_fe',
   'FIELD_47', 'FIELD_25_fe', 'FIELD_16', 'FIELD_46', 'FIELD_45_fe',
   'FIELD_44_fe', 
    "FIELD_30_fe", "FIELD_50"
#     "province", "district", "maCv",
]
target_col = "label"

In [ ]:
train_df["maCv2"] = train_df["maCv"].str.lower()
test_df["maCv2"] = test_df["maCv"].str.lower()

def job_category(x):
    if type(x) == str:
        if "công nhân" in x or "cnv" in x or "thợ may" in x or "cn" in x or "may công nghiệp" in x \
        or "thợ sơn" in x or "coõng nhaõn trửùc tieỏp maựy may coõng nghieọp" in x:
            return "CN"
        elif "giáo viên" in x:
            return "GV"
        elif "nhân viên" in x or "kế toán" in x or "cán bộ" in x or "nv" in x:
            return "NV"
        elif "tài xế" in x or "lái xe" in x:
            return "TX"
        elif "undefined" in x:
            return np.nan
        else:
            return x
    else:
        return x
    
train_df["maCv2"] = train_df["maCv2"].apply(job_category)
test_df["maCv2"] = test_df["maCv2"].apply(job_category)

In [ ]:
for col in cat_cols:
    train_df[col] = train_df[col].astype(str)
    test_df[col] = test_df[col].astype(str)
    
from feature_engine import categorical_encoders as ce

# Label-Encoding 
lbl = LabelEncoder()
for col in cat_cols:
    assert train_df[col].dtypes == test_df[col].dtypes
    
    encoder = ce.CountFrequencyCategoricalEncoder(encoding_method='frequency')
    
    train_df[col] = train_df[col].fillna('None')
    test_df[col] = test_df[col].fillna('None')
    # Only take the common values in Train/Test-set
    common_vals = list(set(train_df[col]).intersection(set(test_df[col])))
    
    # Take if vals appeared both 5 times
    common_vals = set(train_df[col].value_counts()[train_df[col].value_counts()> 20].index)\
                        .intersection(test_df[col].value_counts()[test_df[col].value_counts()>20].index)
    
    # Replace not-common values with "Missing" or np.NaN      
    train_df.loc[~train_df[col].isin(common_vals), col] = 'Missing'
    test_df.loc[~test_df[col].isin(common_vals), col] = 'Missing'

    # Implement LE
    lbl.fit(train_df[col].tolist() + test_df[col].tolist())
    train_df[col] = lbl.transform(train_df[col])
    test_df[col] = lbl.transform(test_df[col])

In [ ]:
fe_cols = cat_cols
for col in fe_cols:
    temp_df = pd.concat([train_df[col], test_df[col]], axis=0)
    d_fe = temp_df.value_counts().to_dict()
    train_df[col+'_fe'] = train_df[col].apply(lambda x: d_fe.get(x))
    test_df[col+'_fe'] = test_df[col].apply(lambda x: d_fe.get(x))

In [ ]:
# How many years passed from the certain action
train_df['passed_year'] = (train_df['FIELD_3']-337)//365
test_df['passed_year'] = (test_df['FIELD_3']-337)//365

# How many years passed from the certain action
train_df['field_3_log'] = train_df['FIELD_3'].apply(np.log1p)
test_df['field_3_log'] = test_df['FIELD_3'].apply(np.log1p)

# train_df['field_51_log'] = train_df['FIELD_51'].apply(np.log1p)
# test_df['field_51_log'] = test_df['FIELD_51'].apply(np.log1p)

In [ ]:
# train_df['is_age_same'] = (train_df['age_source1'] == train_df['age_source2']).astype(np.int8)
# test_df['is_age_same'] = (test_df['age_source1'] == test_df['age_source2']).astype(np.int8)

In [ ]:
train_df.fillna(-1, inplace=True)
test_df.fillna(-1, inplace=True)

train_df.replace("None", -1, inplace=True)
test_df.replace("None", -1, inplace=True)

In [ ]:
train_df.shape

In [ ]:
train_df["FIELD_11"] = train_df["FIELD_11"].astype(float)
test_df["FIELD_11"] = test_df["FIELD_11"].astype(float)

train_df["FIELD_11_22_ratio"] = train_df["FIELD_11"].astype(float) / train_df["FIELD_22"]
test_df["FIELD_11_22_ratio"] = test_df["FIELD_11"].astype(float) / test_df["FIELD_22"]

# train_df["age_ratio"] = train_df["age_source1"] / train_df["age_source2"]
# test_df["age_ratio"] = test_df["age_source1"] / test_df["age_source2"]

In [ ]:
NUM_BOOST_ROUND = 10000

lgbm_param = {'objective':'binary',
              'boosting_type': 'gbdt',
              'metric' : 'auc',
              'learning_rate':0.03,
              'subsample':0.68,
              'tree_learner': 'serial',
              'colsample_bytree':0.28,
              'early_stopping_rounds':50,
              'subsample_freq': 1,
              'reg_lambda':2,
              'reg_alpha': 1,
              'num_leaves': 2 ** 5,
              'random_state':1993,

            }

SEED = 1993
n_splits = 3

In [ ]:
def Gini(y_true, y_pred):
    # check and get number of samples
    assert y_true.shape == y_pred.shape
    n_samples = y_true.shape[0]

    # sort rows on prediction column
    # (from largest to smallest)
    arr = np.array([y_true, y_pred]).transpose()
    true_order = arr[arr[:, 0].argsort()][::-1, 0]
    pred_order = arr[arr[:, 1].argsort()][::-1, 0]

    # get Lorenz curves
    L_true = np.cumsum(true_order) * 1. / np.sum(true_order)
    L_pred = np.cumsum(pred_order) * 1. / np.sum(pred_order)
    L_ones = np.linspace(1 / n_samples, 1, n_samples)

    # get Gini coefficients (area between curves)
    G_true = np.sum(L_ones - L_true)
    G_pred = np.sum(L_ones - L_pred)

    # normalize to true Gini coefficient
    return G_pred * 1. / G_true

In [ ]:
seed_lgbm_oof_train = np.zeros( train_df.shape[0] )
seed_lgbm_final_test =  np.zeros( test_df.shape[0] )
train_label = train_df["label"].values

seeds = range(30)

feature_important = None

cv_list = []
for seed in seeds:
    oof_train = np.zeros( train_df.shape[0] )
    final_test = np.zeros( test_df.shape[0] )

    kfolds = KFold(n_splits=n_splits, random_state=SEED, shuffle=True)
    for ind, (trn_ind, val_ind) in enumerate(kfolds.split(X=train_df, y=train_df["label"])):

        X_train, y_train = train_df.iloc[trn_ind].drop(drop_cols,1), train_label[trn_ind]
        X_valid, y_valid = train_df.iloc[val_ind].drop(drop_cols,1), train_label[val_ind]
        
        ############# TE
        te_cols = ["province", ]
        for col in te_cols:
            te_d = X_train.groupby(col)[target_col].mean().to_dict()
            X_train[col+'_te'] = X_train[col].apply(lambda x: te_d.get(x))
            X_valid[col+'_te'] = X_valid[col].apply(lambda x: te_d.get(x))
            test_df[col+'_te'] = test_df[col].apply(lambda x: te_d.get(x))
            
        X_train = X_train.drop([target_col] ,1)
        X_valid = X_valid.drop([target_col],1)

        dtrain = lgbm.Dataset( X_train, y_train )
        dvalid = lgbm.Dataset( X_valid, y_valid)
        
        lgbm_param['random_state'] = seed

        model = lgbm.train(
            lgbm_param, 
            dtrain, 
            NUM_BOOST_ROUND, 
            valid_sets=(dtrain, dvalid), 
            valid_names=('train','valid'), 
#             categorical_feature=cat_cols,
            verbose_eval= 100
        )

        valid_pred = model.predict(X_valid)
        test_pred  = model.predict(test_df.drop(drop_cols,1))

        oof_train[val_ind] += valid_pred
        final_test += test_pred

        cv_list.append(Gini(y_valid, valid_pred))
        
        if feature_important is None:
            feature_important = model.feature_importance()
        else:
            feature_important += model.feature_importance()

        print('='*80)

    final_test /= n_splits

    # print(f"Average CV : {np.mean(cv_list)}")
    print(f"Gini for OOF: {Gini(train_label, oof_train)}")
    
    
    seed_lgbm_oof_train += oof_train
    seed_lgbm_final_test += final_test
    
seed_lgbm_oof_train /= len(seeds)
seed_lgbm_final_test /= len(seeds)
feature_important = feature_important / (len(seeds) * n_splits)

print(f"Average CV : {np.mean(cv_list)}")
print(f"STD CV : {np.std(cv_list)}")
print(f"Gini for OOF: {Gini(train_label, seed_lgbm_oof_train)}")

In [ ]:
df_imp = pd.DataFrame(data = {'col' : model.feature_name(), 'imp' : feature_important})
df_imp = df_imp.sort_values(by='imp', ascending=False).reset_index(drop=True)
df_imp.tail(50)

In [ ]:
sub = pd.DataFrame({
    "id": test_df["id"],
    "label": seed_lgbm_final_test
})
sub.to_csv("baseline_lgbm.csv", index=False)